# Coursework: Optimisation of a fantasy football team

The coursework is described in detail in the documentation provided on Moodle. This notebook contains some code for basic functions that read in the data file and define the solution/constraint checker that you must use to check your final solution.

As noted in the coursework, you don't have to use Python or DEAP to tackle this. However, the practicals have covered a lot of functionality that will be useful so you should find that the DEAP libraries provide a quick way to start and will save you some time in writing code.

## Important Information

If you use another language, then you should write out your solution to a csv file as a comma separated list of 0,1s (one value per row) indicating which players are included, and use the code provided in this notebook to read it in and check it. You report should include the screenshot of the  output from the function provided in this notebook, and *not* your own version of the function



# Data
The code below reads in the datafile and calculates the number of players available.  
Change the filepath to your local drive.

The file is sorted by player type. As I may check your solution **DO NOT** sort the file or alter it in any way as my code will expect to see it in this format.

Feel free to browse the file and analyse the data in any way you think might be useful

In [1]:
#import some standard python packages that will be useful
import array
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


# import deap packages required
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import pandas as pd


In [2]:
# THIS FUNCTION READS THE DATA FILE CONTAINING THE INFORMATION RE EACH PLAYER

# read data
data = (pd.read_csv("clean-data.csv")
        .reset_index(drop=True))

global num_players
num_players = len(data.index)

print("num possible players is %s" % (num_players))


num possible players is 523


# Helpful data
The code below extracts some useful information from the data that will be useful to you when writing your program. In particular:

- a list containing the **points** per player:  e.g. points[i] refers to the **points** associated with player *i*
- a list containing the **cost** per player: e.g. cost[i] refers to the **cost** associated with player *i*
- a list **gk** which indicates which player is a *goal-keeper*. The list is the same length as the number of players. gk[i]=0 if player *i* is not a goal-keeper; gk[i]=1 if player *i* is a goal-keeper
- a list **mid** which indicates which player is a *midfielder*. The list is the same length as the number of players. mid[i]=0 if player *i* is not a mid-fielder; mid[i]=1 if player *i* is a midfielder
- a list **defe** which indicates which player is a *defender*. The list is the same length as the number of players. defe[i]=0 if player *i* is not a defender; defe[i]=1 if player *i* is a defender
- a list **stri** which indicates which player is a *striker*. The list is the same length as the number of players. stri[i]=0 if player *i* is not a striker; stri[i]=1 if player *i* is a striker

In [3]:
# HELPFUL DATA 
# these can be used for calculating points and costs and are also used in the constraint_checking function
points = data['Points'] 
cost = data['Cost']
    

# create lists with all elements initialised to 0
gk = np.zeros(num_players)
mid = np.zeros(num_players)
defe = np.zeros(num_players)
stri = np.zeros(num_players)

for i in range(num_players):
    if data['Position'][i] == 'GK':
        gk[i] = 1
    elif data['Position'][i] == 'DEF':
        defe[i] = 1
    elif data['Position'][i] == 'MID':
        mid[i] = 1
    elif data['Position'][i] == 'STR':
        stri[i]=1
  

# Solution and constraint checker function

You are free to represent an individiual in any way you wish. However, at the end of the evolutionary run, you *must* convert your solution to a list of length *num_players* in which each element is either 0 or 1. An element *i* should be set to 0 if player *i* is not included in the team, and to 1 if player *is* **is** included in the team.

You *must* call this function with your best solution and include a screen shot of the output in your report.

In [4]:
# check the constraints
# the function MUST be passed a list of length num_players in which each bit is set to 0 or 1


def check_constraints(individual):
     
    broken_constraints = 0

    # exactly 11 players
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints+=1
        print("total players is %s " %(c1))
        
    
    #need cost <= 100"
    c2 = np.sum(np.multiply(cost, individual)) 
    if c2 > 100:
        broken_constraints+=1
        print("cost is %s " %(c2))
    
    # need only 1 GK
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints+=1
        print("goalies is %s " %(c3))
    
    # need less than 3-5 DEF"
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints+=1
        print("DEFE is %s " %(c4))
            
    #need 3- 5 MID
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3: 
        broken_constraints+=1
        print("MID is %s " %(c5))
        
    # need 1 -1 3 STR"
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1: 
        broken_constraints+=1
        print("STR is %s " %(c6))
        
    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]
    
    totalpoints = np.sum(np.multiply(points, individual))
        
        
    print("total broken constraints: %s" %(broken_constraints))
    print("total points: %s" %(totalpoints))
    print("total cost is %s" %(c2))
    print("selected players are %s" %(selectedPlayers))
    
    return broken_constraints, totalpoints

In [5]:
global team_size
team_size = 11
# permutation  approach

global list_of_goal_keepers
list_of_goal_keepers = []
global list_of_defenders
list_of_defenders = []
global list_of_midfielders
list_of_midfielders = []
global list_of_strikers
list_of_strikers = []
# Create individuals as integer permutation representation
def get_positions_of_players(num_players):
    for i in range(num_players):
        # Lists start from index 0 so direct representation would be i+1
        if data['Position'][i] == 'GK':
            list_of_goal_keepers.append(i)
        elif data['Position'][i] == 'DEF':
            list_of_defenders.append(i)
        elif data['Position'][i] == 'MID':
            list_of_midfielders.append(i)
        elif data['Position'][i] == 'STR':
            list_of_strikers.append(i)

In [6]:
# check the constraints
# the function MUST be passed a list of length num_players in which each bit is set to 0 or 1


def check_constraints_modified(individual):

    # Do not need to check if there are duplicates as the initialization takes care of it
    broken_constraints = [0,0,0,0,0,0]
    broken_constraints_num = 0

    # exactly 11 players
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints[0] = 1
        broken_constraints_num += 1
        print("total players is %s " %(c1))


    #need cost <= 100"
    c2 = np.sum(np.multiply(cost, individual))
    if c2 > 100:
        broken_constraints[1] = 1
        broken_constraints_num += 1
        print("cost is %s " %(c2))

    # need only 1 GK
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints[2] = 1
        broken_constraints_num += 1
        print("goalies is %s " %(c3))

    # need less than 3-5 DEF"
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints[3] = 1
        broken_constraints_num += 1
        print("DEFE is %s " %(c4))

    #need 3- 5 MID
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3:
        broken_constraints[4] = 1
        broken_constraints_num += 1
        print("MID is %s " %(c5))

    # need 1 -1 3 STR"
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1:
        broken_constraints[5] = 1
        broken_constraints_num += 1
        print("STR is %s " %(c6))

    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]

    totalpoints = np.sum(np.multiply(points, individual))


    print("total broken constraints: %s" %(broken_constraints_num))
    print("total points: %s" %(totalpoints))
    print("total cost is %s" %(c2))
    print("selected players are %s" %(selectedPlayers))

    return broken_constraints, totalpoints

def check_constraints_modified_no_print(individual):

    # Do not need to check if there are duplicates as the initialization takes care of it
    broken_constraints = [0,0,0,0,0,0]
    broken_constraints_num = 0

    # exactly 11 players - 0
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints[0] = 1
        broken_constraints_num += 1


    #need cost <= 100" - 1
    c2 = np.sum(np.multiply(cost, individual))
    if c2 > 100:
        broken_constraints[1] = 1
        broken_constraints_num += 1

    # need only 1 GK - 2 
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints[2] = 1
        broken_constraints_num += 1

    # need less than 3-5 DEF" - 3 
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints[3] = 1
        broken_constraints_num += 1

    #need 3- 5 MID - 4
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3:
        broken_constraints[4] = 1
        broken_constraints_num += 1

    # need 1 -1 3 STR" - 5
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1:
        broken_constraints[5] = 1
        broken_constraints_num += 1

    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]

    totalpoints = np.sum(np.multiply(points, individual))

    return broken_constraints, totalpoints

In [7]:
global another_representation
another_representation = {}
# Permutation
def initialization_create_feasible_individual(icls, size, pInit):
    try:
        list_of_used_players = []
        # first create an individual with all bits set to 0
        ind = icls(np.zeros(size))
        broken_constraint_array = [1]*5
        #  There must be exactly 11 players in the team
        item_indices = [-1]*11  # individual has to contain exactly 11 players
        # The total cost of the team must be less than or equal to £100
        #  You can’t pick the same player more than once (i.e. all players in a team are unique)
        pos = 0
        while 1 in broken_constraint_array:
            if pos == 11:
                print("OMG")
            broken_constraint_array, totalpoints = check_constraints_runtime(team_size, size, item_indices, icls)
            if 1 not in broken_constraint_array:
                break
            if broken_constraint_array[2]:
                # Need exactly 1 GK
                random.shuffle(list_of_goal_keepers)
                while list_of_goal_keepers[0] in list_of_used_players:
                    random.shuffle(list_of_goal_keepers)
                item_indices[pos]=list_of_goal_keepers[0]
                list_of_used_players.append(item_indices[pos])
                pos += 1
            if broken_constraint_array[3]:
                # Need at least 3 DEF (up to 5)
                for iteration1 in range(3):
                    while list_of_defenders[iteration1] in list_of_used_players:
                        random.shuffle(list_of_defenders)
                    item_indices[pos] = list_of_defenders[iteration1]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
            if broken_constraint_array[4]:
                # Need at least 3 MIN (up to 5)
                random.shuffle(list_of_midfielders)
                for iteration2 in range(3):
                    while list_of_midfielders[iteration2] in list_of_used_players:
                         random.shuffle(list_of_midfielders)
                    item_indices[pos] = list_of_midfielders[iteration2]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
            if broken_constraint_array[5]:
                # Need at least 1 STR (up to 3)
                random.shuffle(list_of_strikers)
                while list_of_strikers[0] in list_of_used_players:
                    random.shuffle(list_of_strikers)
                item_indices[pos] = list_of_strikers[0]
                list_of_used_players.append(item_indices[pos])
                pos += 1
            if broken_constraint_array[0]:
                list_of_choices = []
                list_showing_players = []
                for j in range(11):
                    if item_indices[j] in list_of_goal_keepers:
                        list_showing_players.append("GK")
                    if item_indices[j] in list_of_defenders:
                        list_showing_players.append("DEF")
                    if item_indices[j] in list_of_midfielders:
                        list_showing_players.append("MIN")
                    if item_indices[j] in list_of_strikers:
                        list_showing_players.append("STR")
                def_players = list_showing_players.count("DEF")
                min_players = list_showing_players.count("MIN")
                str_players = list_showing_players.count("STR")
                if def_players < 5:
                    list_of_choices.append(1)
                if min_players < 5:
                    list_of_choices.append(2)
                if str_players < 3:
                    list_of_choices.append(3)
                random.shuffle(list_of_choices)
                choice = list_of_choices[0]
                if choice == 1:
                    random.shuffle(list_of_defenders)
                    while list_of_defenders[0] in list_of_used_players:
                        random.shuffle(list_of_defenders)
                    item_indices[pos] = list_of_defenders[0]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
                elif choice == 2:
                    random.shuffle(list_of_midfielders)
                    while list_of_midfielders[0] in list_of_used_players:
                        random.shuffle(list_of_midfielders)
                    item_indices[pos] = list_of_midfielders[0]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
                elif choice == 3:
                    random.shuffle(list_of_strikers)
                    while list_of_strikers[0] in list_of_used_players:
                        random.shuffle(list_of_strikers)
                    item_indices[pos] = list_of_strikers[0]
                    list_of_used_players.append(item_indices[pos])
                    pos += 1
                if 1 is not broken_constraint_array[0] is not broken_constraint_array[2] is broken_constraint_array[3] and 1 is not broken_constraint_array[4] and 1 is not broken_constraint_array[5]:
                    break
        item_indices_ind = [0]*size
        for i in range(team_size):
                item = item_indices[i]
                if item != -1:
                    ind[item]=1
                    item_indices_ind[item] = 1
        s = ','.join(str(x) for x in item_indices_ind)
        another_representation.update({s : item_indices})
        return ind
    except IndexError as e:
        list_showing_players = []
        for j in range(11):
            if item_indices[j] in list_of_goal_keepers:
                list_showing_players.append("GK")
            if item_indices[j] in list_of_defenders:
                list_showing_players.append("DEF")
            if item_indices[j] in list_of_midfielders:
                list_showing_players.append("MIN")
            if item_indices[j] in list_of_strikers:
                list_showing_players.append("STR")
        print(f'caught {type(e)}: e')

    
def check_constraints_runtime(team_size, size, item_indices, icls):
     # first create an individual with all bits set to 0
    ind = icls(np.zeros(size))
    for i in range(team_size):
            item = item_indices[i]
            if item != -1:
                ind[item]=1
    broken_constraint_array, totalpoints = check_constraints_modified(ind)
    return broken_constraint_array, totalpoints

<>:98: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:98: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:98: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:98: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:98: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:98: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\venet\AppData\Local\Temp\ipykernel_8932\3517117681.py:98: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if 1 is not broken_constraint_array[0] is not broken_constraint_array[2] is broken_constraint_array[3] and 1 is not broken_constraint_array[4] and 1 is not broken_constraint_array[5]:
C:\Users\venet\AppData\Local\Temp\ipykernel_8932\3517117681.py:98: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if 1 is not broken_constraint_array[0] is not broken_constraint_array[2] is broken_constraint_array[3] and 1 is not broken_constraint_array[4] and 1 is not broken_c

In [8]:
# https://github.com/fergaljd/pep_ga/blob/d7f0d5877009ea0bb2ff452d2454ba1e0ebfb4a1/selection.py
def _roulette_wheel(population, pointers):
    keep = []
    i = 0
    cum_fitness = population[i].fitness.values[0]
    for point in pointers:
        while cum_fitness < point:
            i += 1
            cum_fitness += population[i].fitness.values[0]
        keep.append(population[i])
    #Picks should be unique!
    
    return keep


In [9]:
# Selection method
# https://github.com/fergaljd/pep_ga/blob/d7f0d5877009ea0bb2ff452d2454ba1e0ebfb4a1/selection.py            
def stochastic_universal_sampling(population, individuals, variable_2):
    '''
    Similar to the proportional selection,
    every individual obtains a segment on a
    roulette wheel according to its fitness
    value.
    However, it is turned only one time with
    nballs where n is the number of individuals in the
    population.
    https://en.wikipedia.org/wiki/Stochastic_universal_sampling
    '''
    number_to_keep=4
    sum_fitnesses = sum([c.fitness.values[0] for c in individuals])
    pointer_spread = float(sum_fitnesses) / number_to_keep
    start = random.uniform(0, pointer_spread)
    pointers = [(start+i*pointer_spread) for i in range(number_to_keep)]
    return _roulette_wheel(individuals, pointers)

In [20]:
# Crossover - to do
def cxTwoPoint_(ind1, ind2):
    """Executes a two-point crossover on the input :term:`sequence`
    individuals. The two individuals are modified in place and both keep
    their original length.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.randint` function from the Python
    base :mod:`random` module.
    """
    # 0 - 180 DEF -> from 3 to 5 players
    def_to_crossover_1 = []
    def_to_crossover_2 = []
    # 181 - 376 MID
    mid_to_crossover_1 = []
    mid_to_crossover_2 = []
    # 377 - 466 STR
    str_to_crossover_1 = []
    str_to_crossover_2 = []
    # 467 - 523 GK
    GK_to_crossover_1 = []
    GK_to_crossover_2 = []
    # get indices of players selected
    selectedPlayers_individual_1 = [idx for idx, element in enumerate(ind1) if element==1]
    selectedPlayers_individual_2 = [idx for idx, element in enumerate(ind2) if element==1]

    def_to_crossover_1,mid_to_crossover_1, str_to_crossover_1, GK_to_crossover_1 = select_genomes_to_crossover(selectedPlayers_individual_1, def_to_crossover_1,mid_to_crossover_1, str_to_crossover_1, GK_to_crossover_1)
    def_to_crossover_2,mid_to_crossover_2, str_to_crossover_2, GK_to_crossover_2 = select_genomes_to_crossover(selectedPlayers_individual_2, def_to_crossover_2,mid_to_crossover_2, str_to_crossover_2, GK_to_crossover_2)

    size_def = min(len(def_to_crossover_1), len(def_to_crossover_2))
    size_mid = min(len(mid_to_crossover_1), len(mid_to_crossover_2))
    size_str = min(len(str_to_crossover_1), len(str_to_crossover_2))
    size_gk = min(len(GK_to_crossover_1), len(GK_to_crossover_2))

    cxpoint_def = random.randint(0, size_def)
    cxpoint_mid = random.randint(0, size_mid)
    cxpoint_str = random.randint(0, size_str)
    cxpoint_gk = random.randint(0, size_gk)

    

    size = min(len(ind1), len(ind2))
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]

    return ind1, ind2
def select_genomes_to_crossover(individual, def_to_crossover,mid_to_crossover, str_to_crossover, GK_to_crossover):
    # 
    for genome in individual:
        if genome >= 0 and  genome <= 180 :
            def_to_crossover.append(genome)
        elif genome >= 181 and  genome <= 376:
            mid_to_crossover.append(genome)
        elif genome >= 377 and  genome <= 466 :
            str_to_crossover.append(genome)
        elif genome >= 467 and  genome <= 523 :
            GK_to_crossover.append(genome)
    return def_to_crossover,mid_to_crossover, str_to_crossover, GK_to_crossover

In [11]:
# DEFINE FITNESS FOR KNAPSACK
# fitness function definition - death penalty

def evalKnapsack1(individual):
    cost_for_individual = np.sum(np.multiply(cost, individual))
    points_for_individual = np.sum(np.multiply(points, individual))
    if  cost_for_individual > MAX_Cost:
        total_overbudget = cost_for_individual - MAX_Cost        # Bags that are overweight get a fitness of difference to max cost
        return points_for_individual - total_overbudget,
    return  points_for_individual,

In [21]:
# this returns a single individual: this function has the probability pInit of initialsing as feasible:
# if it is set to 0, initialisation is all random. If it is 1, initialistion is all feasible
# Binary Representation
MAX_Cost = 100
global TOTAL_WEIGHT
TOTAL_WEIGHT = 0
for i in range(num_players):
    TOTAL_WEIGHT += points[i]

global TOTAL_PROFIT
TOTAL_PROFIT = 0
for i in range(num_players):
    TOTAL_PROFIT += cost[i]

get_positions_of_players(num_players)
# create a toolbox
toolbox = base.Toolbox()
# define the fitness class and creare an individual class
creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # Maximization problem for the value not the cost
creator.create("Individual", list, fitness=creator.FitnessMax)
# USE THIS LINE IF YOU WANT TO USE THE CUSTOM INIT FUNCTION
toolbox.register("individual", initialization_create_feasible_individual, creator.Individual, num_players, 1.0)

#  a population consist of a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# register all operators we need with the toolbox
toolbox.register("evaluate", evalKnapsack1)
toolbox.register("mate", cxTwoPoint_)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", stochastic_universal_sampling, toolbox.population)
pop = toolbox.population(n=100)
for individual in pop:
    broken_constraint, total_poins = check_constraints_modified_no_print(individual)
    if broken_constraint.count(1) > 2:
        print("Problem")
    else:
        print("No PROBLEM")
# keep track of the single best solution found
hof = tools.HallOfFame(1)

# create a statistics object: we can log what ever statistics we want using this. We use the numpy Python library
# to calculate the stats and label them with convenient labels
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# run the algorithm: we need to tell it what parameters to use
# cxpb = crossover probability; mutpb = mutation probability; ngen = number of iterations
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.01, ngen=400,
                               stats=stats, halloffame=hof, verbose=True)

best = hof[0].fitness.values[0]   # best fitness found is stored at index 0 in the hof list


# look in the logbook to see what generation this was found at
max = log.select("max")  # max fitness per generation stored in log
sorted_pop = sorted(pop, key=lambda ind: ind.fitness, reverse=True)
print(sorted_pop[0])
check_constraints(sorted_pop[0])
for i in range(200):  # set to ngen
    fit = max[i]
    if fit == best:
        break

print("max fitness found is %s at generation %s" % (best, i))

c:\Users\venet\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\venet\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


total players is 0.0 
goalies is 0.0 
DEFE is 0.0 
MID is 0.0 
STR is 0.0 
total broken constraints: 5
total points: 0.0
total cost is 0.0
selected players are []
total players is 9.0 
total broken constraints: 1
total points: 328.0
total cost is 63.8
selected players are [71, 83, 87, 147, 330, 341, 345, 465, 475]
total players is 10.0 
total broken constraints: 1
total points: 336.0
total cost is 71.3
selected players are [71, 83, 87, 147, 330, 341, 345, 438, 465, 475]
OMG
total broken constraints: 0
total points: 372.0
total cost is 79.1
selected players are [71, 83, 87, 147, 330, 341, 345, 423, 438, 465, 475]
total players is 0.0 
goalies is 0.0 
DEFE is 0.0 
MID is 0.0 
STR is 0.0 
total broken constraints: 5
total points: 0.0
total cost is 0.0
selected players are []
total players is 9.0 
total broken constraints: 1
total points: 335.0
total cost is 66.1
selected players are [147, 174, 178, 221, 237, 255, 323, 411, 510]
total players is 10.0 
total broken constraints: 1
total poin

TypeError: '<=' not supported between instances of 'tuple' and 'list'